# odc-stac tutorial

In this tutorial we will use Python libraries to find and load land cover data from the freely available [Impact Observatory Annual Land Use Land Cover](https://planetarycomputer.microsoft.com/dataset/io-lulc-annual-v02) product.
After loading the data, we will export it as a cloud-optimised GeoTiff.
This will allow you to further view or work with the data in GIS software and other tools.

During the tutorial, we will:

* Specify our search in terms of:

  * what (data provider and satellite)
  * where (area of interest)
  * when (date range)
* Use `pystac-client` to connect to a Spatio-Temporal Asset Catalog (STAC) 
  endpoint and search for data matching our what, where, and when
* Use `odc-stac` to load the matching data into memory
* Visualise and export the data

## Requirements
Please keep a copy of the [tutorial instructions](https://opendatacube.readthedocs.io/en/latest/tutorials/odc-stac.html) open.

Follow the instructions for each labelled section. 
When you have finished entering code into a cell, press `Shift+Enter` on your keyboard to run the cell.

## Python imports

In [1]:
import geopandas as gpd
from odc.stac import load
from pystac_client import Client

## Set up query parameters

### Area of interest

In [2]:
aoi = gpd.read_file("aoi.geojson")

aoi_geometry = aoi.iloc[0].geometry

### Date range

In [3]:
start_date = "2021-12-24"
end_date = "2021-12-26"

date_query = (start_date, end_date)

### Catalog and measurements

You can use the Radiant Earth [stac-browser website](https://radiantearth.github.io/stac-browser/#/?.language=en) to preview catalog, collection, and item information.



#### Catalog
![alt text](images/catalog.png "Earth Search by Element 84")

The overarching structure that is used to organise satellite datasets and their individual images.

Example: [earth-search.aws.element84.com](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/)

#### Collection
![alt text](images/collection.png "Sentinel-2 Collection")

The structure used to organise individual datasets.

Example: [sentinel-2-l2a](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-l2a)


#### Item
![alt text](images/item.png "Sentinel-2 Collection")

A single image from the collection. The image may have multiple assets (bands).

Example: [S2B_52KDA_20211225_0_L2A](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_52KDA_20211225_0_L2A)


In [4]:
catalog = "https://earth-search.aws.element84.com/v1/"
collections_query = ["sentinel-2-l2a"]
bands_query = ["red", "green", "blue"]

## Connect to catalog and find items

This step returns STAC items that match the query, each of which points to a Cloud Optimised GeoTIFF (COG) that contains the relevant data

In [5]:
# Connect to the catalog
stac_client = Client.open(catalog)

### Search for items

`pystac-client` is used to search for items that match the query. The search function has a few arguments:

- `collections`: a list of collections to search
- `intersects`: an area of interest to search. Any item intersecting the geometry will be returned.
- `datetime`: a date range to search

In [6]:
# Search for items matching the query
items = stac_client.search(
    collections=collections_query,
    intersects=aoi_geometry,
    datetime=date_query,
).item_collection()

print(f"Found {len(items)} items")

Found 4 items


Once returned, it is possible to view relevant metadata for all discovered items.

In [7]:
items

## Load items with odc-stac

Once the location of the items is known, `odc-stac` can load these items. The load function has a few arguments:

- `items`: a list of STAC items to load
- `bands`: a list of assets to load for each item
- `geopolygon`: the area of interest to load the data for
- `crs`: the coordinate reference system to use. `utm` will provide the most appropriate UTM projection
- `resolution`: the resolution to load the items at, in the same units as the `crs`. If using `utm` the unit is metres
- `groupby`: using `solar_day` here will merge all items captured on the same solar day into one image


In [8]:
ds_filtered = load(
    items,
    bands=bands_query,
    geopolygon=aoi_geometry,
    crs="utm",
    resolution=30,
    groupby="solar_day",
)

ds_filtered

<xarray.Dataset> Size: 3MB
Dimensions:      (y: 562, x: 878, time: 1)
Coordinates:
  * y            (y) float64 4kB 7.568e+06 7.568e+06 ... 7.551e+06 7.551e+06
  * x            (x) float64 7kB 4.947e+05 4.947e+05 ... 5.209e+05 5.21e+05
    spatial_ref  int32 4B 32752
  * time         (time) datetime64[ns] 8B 2021-12-25T01:43:25.628000
Data variables:
    red          (time, y, x) uint16 987kB 2143 2165 2124 ... 1736 1574 1578
    green        (time, y, x) uint16 987kB 1012 1039 1011 968 ... 939 869 843
    blue         (time, y, x) uint16 987kB 643 660 660 631 ... 567 608 568 532

## Visualise loaded data

If an `xarray` has been loaded using the Open Data Cube ecosystem, it comes with an `xarray` extension `xarray.odc` that allows you to access additional geospatial metadata and functions.

Learn more at [odc-geo](https://odc-geo.readthedocs.io/en/latest/intro-xr.html).

In [9]:
# Get a single image at the first time step, and remove the time dimension
image = ds_filtered.isel(time=0).squeeze()

image

<xarray.Dataset> Size: 3MB
Dimensions:      (y: 562, x: 878)
Coordinates:
  * y            (y) float64 4kB 7.568e+06 7.568e+06 ... 7.551e+06 7.551e+06
  * x            (x) float64 7kB 4.947e+05 4.947e+05 ... 5.209e+05 5.21e+05
    spatial_ref  int32 4B 32752
    time         datetime64[ns] 8B 2021-12-25T01:43:25.628000
Data variables:
    red          (y, x) uint16 987kB 2143 2165 2124 2059 ... 1580 1736 1574 1578
    green        (y, x) uint16 987kB 1012 1039 1011 968 971 ... 890 939 869 843
    blue         (y, x) uint16 987kB 643 660 660 631 631 ... 530 567 608 568 532

In [10]:
# Create an RGB image using the .odc. extension
visualisation = image.odc.to_rgba()

# Display the RGB image on a map
visualisation.odc.explore()

## Export loaded 

The `.odc` extension is also useful for saving out data as Cloud Optimised GeoTIFFs (COGs).

In [11]:
visualisation.odc.write_cog(
    "sentinel2_example.tif",
    overwrite=True
)

PosixPath('sentinel2_example.tif')

## Tutorial Complete!

Try loading your saved COG in QGIS or another GIS program.